In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

C:\Users\Satyen\anaconda3\envs\py3XD\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Load in Images From Folder

In [2]:
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import os
# current_path = os.getcwd()
# print(current_path)

dir_path = 'C:\\Users\\Satyen\\Documents\\College\\Fall 2023\\Data Science\\Final Project\\Ingredients\\train'

train_data = []

class_label_arr = []
for class_label in os.listdir(dir_path):
    class_label_arr.append(class_label)
# image

Preprocessing

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_path = 'C:\\Users\\Satyen\\Documents\\College\\Fall 2023\\Data Science\\Final Project\\Ingredients\\train'

validation_path = 'C:\\Users\\Satyen\\Documents\\College\\Fall 2023\\Data Science\\Final Project\\Ingredients\\validation'

tg_height, tg_width = 500,500

train_dataGen = ImageDataGenerator(rescale=1./255)  # Normalize pixel values)

validation_dataGen = ImageDataGenerator(rescale=1./255)

train_gen = train_dataGen.flow_from_directory(
    train_path,
    target_size=(tg_height, tg_width),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

validation_gen = validation_dataGen.flow_from_directory(
    validation_path,
    target_size=(tg_height, tg_width),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)



Found 3157 images belonging to 37 classes.
Found 361 images belonging to 37 classes.


In [4]:
print(train_gen.num_classes)

37


In [5]:
cnn = tf.keras.models.Sequential([

    #feature extraction
    layers.Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', input_shape=(500,500,3)),
    layers.MaxPooling2D((4,4)),
    layers.Conv2D(filters = 64, kernel_size = (2,2), activation = 'relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(filters = 64, kernel_size = (2,2), activation = 'relu'),
    layers.MaxPooling2D((4,4)),
    layers.Dropout(0.3),

    layers.Flatten(),
    layers.Dense(128, activation = 'relu'),
    layers.Dense(train_gen.num_classes, activation = 'softmax')
])

In [6]:
cnn.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001),
            loss = "categorical_crossentropy",
            metrics = ['accuracy'])

cnn.fit(train_gen, epochs = 15, validation_data = validation_gen, batch_size=64)

Epoch 1/15
15/99 [===>..........................] - ETA: 11:45 - loss: 3.6247 - accuracy: 0.0312

C:\Users\Satyen\anaconda3\envs\py3XD\lib\site-packages\PIL\Image.py:979: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


99/99 [==============================] - 525s 5s/step - loss: 3.2234 - accuracy: 0.1052 - val_loss: 2.5131 - val_accuracy: 0.2576
Epoch 2/15
99/99 [==============================] - 437s 4s/step - loss: 2.4974 - accuracy: 0.2677 - val_loss: 1.8217 - val_accuracy: 0.4848
Epoch 3/15
99/99 [==============================] - 542s 5s/step - loss: 2.0063 - accuracy: 0.4131 - val_loss: 1.3041 - val_accuracy: 0.6233
Epoch 4/15
99/99 [==============================] - 464s 5s/step - loss: 1.6076 - accuracy: 0.5283 - val_loss: 0.8198 - val_accuracy: 0.7839
Epoch 5/15
99/99 [==============================] - 492s 5s/step - loss: 1.1640 - accuracy: 0.6598 - val_loss: 0.5381 - val_accuracy: 0.8753
Epoch 6/15
99/99 [==============================] - 483s 5s/step - loss: 0.8031 - accuracy: 0.7608 - val_loss: 0.4834 - val_accuracy: 0.8837
Epoch 7/15
99/99 [==============================] - 470s 5s/step - loss: 0.5291 - accuracy: 0.8438 - val_loss: 0.3840 - val_accuracy: 0.9003
Epoch 8/15
99/99 [======

In [7]:
test_path = 'C:\\Users\\Satyen\\Documents\\College\\Fall 2023\\Data Science\\Final Project\\Ingredients\\test'



test_gen = validation_dataGen.flow_from_directory(
    test_path,
    target_size=(tg_height, tg_width),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 368 images belonging to 37 classes.


In [8]:
cnn.evaluate(test_gen)

12/12 [==============================] - 34s 3s/step - loss: 0.3782 - accuracy: 0.9429


[0.3781841993331909, 0.9429348111152649]

In [9]:
from sklearn.metrics import classification_report

y_pred = cnn.predict_generator(test_gen, steps=len(test_gen))
y_pred_class = np.argmax(y_pred, axis=1)
y_actual = test_gen.classes

rep = classification_report(y_actual, y_pred_class, target_names=test_gen.class_indices.keys())

print(rep)

C:\Users\Satyen\AppData\Local\Temp\ipykernel_10228\852676990.py:3: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  y_pred = cnn.predict_generator(test_gen, steps=len(test_gen))


               precision    recall  f1-score   support

        apple       1.00      0.78      0.88         9
       banana       1.00      0.89      0.94         9
     beetroot       1.00      1.00      1.00        10
  bell pepper       0.91      1.00      0.95        10
      cabbage       1.00      1.00      1.00        10
     capsicum       1.00      0.90      0.95        10
       carrot       0.90      0.90      0.90        10
  cauliflower       0.91      1.00      0.95        10
      chicken       0.75      0.30      0.43        10
chilli pepper       0.90      0.90      0.90        10
         corn       0.83      1.00      0.91        10
     cucumber       1.00      1.00      1.00        10
     eggplant       1.00      1.00      1.00        10
       garlic       0.77      1.00      0.87        10
       ginger       1.00      1.00      1.00        10
       grapes       1.00      1.00      1.00        10
     jalepeno       1.00      1.00      1.00        10
         

In [54]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np


print(class_label_arr)
img_path = 'C:\\Users\\Satyen\\Documents\\College\\Fall 2023\\Data Science\\Final Project\\Ingredients\\predict\\watermelon.jpeg'

image = load_img(img_path, target_size = (500,500))

img_array = img_to_array(image)
# img_array = np.expand_dims(img_array, axis=0)
img_array = np.array([img_array])

#Normalize image
img_array = img_array / 255.0

pred = cnn.predict(img_array)
predicted_class = class_label_arr[np.argmax(pred)]
print(predicted_class)
# img_array = image.img


['apple', 'banana', 'beetroot', 'bell pepper', 'cabbage', 'capsicum', 'carrot', 'cauliflower', 'chicken', 'chilli pepper', 'corn', 'cucumber', 'eggplant', 'garlic', 'ginger', 'grapes', 'jalepeno', 'kiwi', 'lemon', 'lettuce', 'mango', 'onion', 'orange', 'paprika', 'pear', 'peas', 'pineapple', 'pomegranate', 'potato', 'raddish', 'soy beans', 'spinach', 'sweetcorn', 'sweetpotato', 'tomato', 'turnip', 'watermelon']
1/1 [==============================] - 0s 48ms/step
watermelon


In [52]:
cnn.save("ing_detection_model.h5")

NameError: name 'model' is not defined